In [1]:
import os
import re

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import glob


C:\Users\Vicente Laiseca\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
model_dir = 'imagenet'
images_dir = 'images/images_sample/'
folder_list = glob.glob('images/images_sample/*')

In [3]:
% time
image_list = []
for folder in folder_list:
    for image in glob.glob(folder + '/*'):
        image_list.append(image)

Wall time: 0 ns


In [4]:
def create_graph():
    with gfile.FastGFile(os.path.join(
    model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [5]:
def extract_features(list_images):
    nb_features = 2048
    features = np.empty((len(list_images),nb_features))
    labels = []

    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
        print(next_to_last_tensor)

        for ind, image in enumerate(list_images):
            print(ind)
            if (ind):#%100 == 0):
                print('Processing %s...' % (image))
            if not gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)

            image_data = gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor,
            {'DecodeJpeg/contents:0': image_data})

            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',image.split('/')[1])[0])

    return features, labels


In [8]:
%%time40 

Tensor("pool_3:0", shape=(1, 1, 1, 2048), dtype=float32)
0
1
Processing images/images_sample\6811957\6811957_7d3ab8175d23fee64c0651b1bc16f2cc.jpg...
2
Processing images/images_sample\6811957\6811957_83a4e2e75ea15a5a1d2cc9de3407b1b9.jpg...
3
Processing images/images_sample\6811957\6811957_acbdbbe6ff435b9d4f520db6da1ada9a.jpg...
4
Processing images/images_sample\6811957\coche.jpg...
5
Processing images/images_sample\6811957\test.jpg...
6
Processing images/images_sample\6811958\6811958_1fe0076c8b481e0af2223afab02503da.jpg...
7
Processing images/images_sample\6811958\6811958_23ceae42d2c88ffc5a057db1deb346c1.jpg...
8
Processing images/images_sample\6811958\6811958_bb863a4184a1e085f0c55e0172767abd.jpg...
9
Processing images/images_sample\6811958\6811958_c131c57b97afd739161579cdb41c9884.jpg...
10
Processing images/images_sample\6811960\6811960_3685d3542328b820980642535d8ccb72.jpg...
11
Processing images/images_sample\6811960\6811960_a3bd517552d4163c4732c4bd5334b2e0.jpg...
12
Processing images

In [ ]:
pd.DataFrame(features)

In [6]:
pd.DataFrame(features).T.corr()

NameError: name 'features' is not defined

In [ ]:
list_images